Mikes Magic Import Incantation

In [ ]:

from __future__ import print_function, division

import numpy as np
import nilearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%pylab inline

import pandas as pd

import nibabel

from nilearn import plotting

import csv

import os

from IPython.display import Image
from IPython.display import Markdown
from IPython.display import display

%load_ext rpy2.ipython
import rpy2
import rpy2.robjects as robjects
        

In [1]:
import os
import sys
import re
from threading import Thread
from collect_raw_data.collect_raw_data import init_fmri_subject_dir, download_raw_fmri_data
from preprocessing.PreMelodicProcessing import Preprocessing
from preprocessing.StructuralProcessing import StructuralProcessing
from preprocessing.Melodic import Melodic
from preprocessing.PostMelodicProcessing import PostMelodic
from connectome_analyses.FunctionalConnectivity import FunctionalConnectome


class PreprocessingPipeline:
    """ Wraps a subject in the entire preprocessing pipeline
        
        Allows for parellelization and easy management of which
        stages of preprocessing are used
    """

    def __init__(self, subject_list, phase):
        self.subject_list = subject_list
        self.phase = phase.lower()

    def pipeline(self):
        """ Monolithic method that contains the entire preprocessing pipeline

            Stages of preprocessing are specified via command line args:
                Premelodic - initial preprocessing steps on time series,
                             including volume removal, slicetime correction,
                             motion correction, distortion correction,
                             registration into T1 space, etc.
                Melodic - independent component analysis across time series;
                          as of now, hand denoising MUST be done after this step
                Postmelodic - Removal of noise components, spatial smoothing,
                          masking of confounds, and generation of 
                          functional connectome for each subject
        """

        for subjectID in self.subject_list:
            if re.search('[0-9]', subjectID):
                subjectID = check_prefix(subjectID)
                subjectID = subjectID.strip()
            else:
                continue

            # docker container can't see QSM repo, so can't check from notebook
#             if starting_files_present(subjectID) == False:
#                 print("Skipping {}, not all files present".format(subjectID))
#                 continue
#             else:
#                 print(subjectID)

            try:
                print(subjectID)
                if self.phase == "melodic":
                    processing = Preprocessing(subjectID)
                    melodic = Melodic(processing)

                    melodic.init_melodic_directory()
                    melodic.ICA()

                elif self.phase == "postmelodic":
                    processing = Preprocessing(subjectID)
                    postmel = PostMelodic(processing)
                    structproc = StructuralProcessing(processing)
                    fcon = FunctionalConnectome(subjectID)

                    postmel.denoise()
                    print("Removed noise components")
#                     structproc.gm_mask()
#                     structproc.csf_mask()
#                     structproc.wm_mask()
#                     structproc.binarize_masks()
                    postmel.mask_mean_time_series()
                    print("Masked Mean Time Series")
                    postmel.create_all_confounds()
                    print("Created all confounds mask")
                    fcon.create_functional_connectivity_matrix()
                    print("Generated functional connectome")

                elif self.phase == "premelodic":
                    init_fmri_subject_dir(subjectID,
                                          "/shared/studies/nonregulated/connectome/fmri/subjects/",
                                          "/shared/studies/nonregulated/connectome/Subjects/" + subjectID + "/T1w/")

                    processing = Preprocessing(subjectID)
                    structproc = StructuralProcessing(processing)

                    processing.remove_first_two_volumes()
                    processing.slicetime_correction()
                    processing.motion_correction()
                    processing.intensity_normalization()
                    processing.temporal_mean()
                    processing.temporal_filtering()
                    processing.brain_extraction()
                    processing.epi_distortion_correction()
                    processing.zero_center_fieldmap()

                    structproc.generate_aparcaseg()
                    structproc.downsize_T1()

                    processing.ANTs_registration()
                    processing.motion_outlier_detection()
                    processing.spatial_smoothing()

                elif self.phase == "all":
                    init_fmri_subject_dir(subjectID,
                                          "/shared/studies/nonregulated/connectome/fmri/subjects/",
                                          "/shared/studies/nonregulated/connectome/Subjects/" + subjectID + "/T1w/")

                    processing = Preprocessing(subjectID)
                    structproc = StructuralProcessing(processing)
                    # melodic = Melodic(processing)
                    postmel = PostMelodic(processing)

                    processing.remove_first_two_volumes()
                    processing.slicetime_correction()
                    processing.motion_correction()
                    processing.intensity_normalization()
                    processing.temporal_mean()
                    processing.temporal_filtering()
                    processing.brain_extraction()
                    processing.epi_distortion_correction()
                    processing.zero_center_fieldmap()

                    structproc.generate_aparcaseg()
                    structproc.downsize_T1()
                    
                    processing.ANTs_registration()
                    processing.motion_outlier_detection()
                    processing.spatial_smoothing()
                    
                    melodic.init_melodic_directory()
                    melodic.ICA()
                    postmel.denoise()

                    structproc.gm_mask()
                    structproc.csf_mask()
                    structproc.wm_mask()
                    structproc.binarize_masks()

                    postmel.mask_mean_time_series()
                    postmel.create_all_confounds()
                    
                else:
                    processing = Preprocessing(subjectID)
                    fcon = FunctionalConnectome(subjectID)
                    fcon.create_functional_connectivity_matrix()

            except:
                print("{}: Something went wrong".format(subjectID))
                pass

#             break


def starting_files_present(subjectID):
    """ Verify that each subject has:
            Fully processed strucutral image (T1 freesurfer output)
            Unprocessed raw functional image from bluesky
            QSM fieldmap images (magnitude and phase images)
        before beginning processing
    """

    path_conntectome = "/shared/studies/nonregulated/connectome/"
    functional_data = path_conntectome + "fmri/subjects/" + subjectID + "/rawfunc.nii.gz"
    structural_data = path_conntectome + "Subjects/" + subjectID + "/T1w/T1w_acpc_dc_restore_brain.nii.gz"
    QSM_path = "/shared/studies/nonregulated/qsm_repo/data/" + subjectID.replace("EX","") + "/recon/"

    if (not os.path.exists(functional_data)
            or not os.path.exists(structural_data)
            or not os.path.exists(QSM_path + "magnitude_combined.nii.gz")
            or not os.path.exists(QSM_path + "phase_combined.nii.gz")):
        return False
    else:
        return True


def check_prefix(subject):
    """ Append 'EX' to subject's ID if necessary
    """

    if "EX" not in subject:
        subject = "EX" + subject

    return subject


def create_threads(full_subject_list, num_threads):
    """ Split the input subject list into a specified number of sublists
        to parellelize the preprocessing pipeline up to 8 times
    """

    for i in range(0, len(full_subject_list), num_threads):
        yield(full_subject_list[i:i + num_threads])


if __name__ == "__main__":

    # Read in subject list from command line argument
#     with open(sys.argv[1]) as infile:
#         full_subject_list = infile.read().splitlines()
    with open('missing_subs.txt') as infile:
        full_subject_list = infile.read().splitlines()

#     phase = sys.argv[2]
    phase = "postmelodic"
    print(phase)
    preproc = PreprocessingPipeline(full_subject_list, phase)
    preproc.pipeline()

    # Split up list into specified number of sublists and run in parellel
#     num_threads = int(sys.argv[2])
#     phase = sys.argv[3]
#     num_threads = 10
#     phase = "something else"

#     for thread in create_threads(full_subject_list, num_threads):
#        preproc = PreprocessingPipeline(thread, phase)
#        thread_process = Thread(target = preproc.pipeline)
#        thread_process.start()

postmelodic
EX82576
Removed noise components
Masked Mean Time Series
Created all confounds mask


preprocessing/PostMelodicProcessing.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  mc = pd.read_csv(self.path_fmri + '/mc/prefiltered_func_data_mcf.par', sep='  ', header=None)


Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX82576/T1w/GMmask_2.00.nii.gz')
[Memory]    0.0s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min


/usr/local/lib/python2.7/dist-packages/numpy/linalg/linalg.py:1965: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)


Generated functional connectome
EX55777
Removed noise components
Masked Mean Time Series
Created all confounds mask
Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX55777/T1w/GMmask_2.00.nii.gz')
[Memory]    0.0s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min


...

/usr/lib/python2.7/dist-packages/sklearn/covariance/graph_lasso_.py:252: ConvergenceWarning: graph_lasso: did not converge after 100 iteration: dual gap: -2.406e-04
  ConvergenceWarning)


Generated functional connectome
EX84791
Removed noise components
Masked Mean Time Series
Created all confounds mask
Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX84791/T1w/GMmask_2.00.nii.gz')
[Memory]    0.0s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
Generated functional connectome
EX84790
Removed noise components
Masked Mean Time Series
Created all confounds mask
Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX84790/T1w/GMmask_2.00.nii.gz')
[Memory]    0.1s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
Generated functional connectome
EX85404
Removed noise components
Masked Mean Time Series
EX85404: Something went wrong
EX85401
Removed noise components
Mask

/usr/lib/python2.7/dist-packages/sklearn/covariance/graph_lasso_.py:252: ConvergenceWarning: graph_lasso: did not converge after 100 iteration: dual gap: -3.107e-03
  ConvergenceWarning)


Generated functional connectome
EX84143
Removed noise components
Masked Mean Time Series
Created all confounds mask
Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX84143/T1w/GMmask_2.00.nii.gz')
[Memory]    0.1s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
Generated functional connectome
EX84598
Removed noise components
Masked Mean Time Series
Created all confounds mask
Extracted fMRI signal
[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/shared/studies/nonregulated/connectome/Subjects/EX84598/T1w/GMmask_2.00.nii.gz')
[Memory]    0.1s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
Generated functional connectome


/usr/lib/python2.7/dist-packages/sklearn/covariance/graph_lasso_.py:252: ConvergenceWarning: graph_lasso: did not converge after 100 iteration: dual gap: -2.099e-04
  ConvergenceWarning)


In [ ]:
!python main.py missing_subs.txt 7 postmelodic